# Fine-tune FLAN-T5 for chat & dialogue summarization



## 1. Setup Development Environment

Our first step is to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages.

In [1]:
!pip install -q -U accelerate git+https://github.com/huggingface/accelerate.git
!pip install -q -U bitsandbytes
!pip install -q -U datasets==2.13.1
!pip install -q -U scipy
!pip install -q -U evaluate==0.4.0
!pip install -q -U transformers==4.33.3

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7

We want to evaluate our model during training. The `Trainer` supports evaluation during training by providing a `compute_metrics`.  
The most commonly used metrics to evaluate summarization task is [rogue_score](https://en.wikipedia.org/wiki/ROUGE_(metric)) short for Recall-Oriented Understudy for Gisting Evaluation). This metric does not behave like the standard accuracy: it will compare a generated summary against a set of reference summaries

We are going to use `evaluate` library to evaluate the `rogue` score.

In [ ]:
!pip install rouge_score==0.1.2

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=e6ae06e9b52aad4011177a0ff00d6a135c8c039d37fbe064dc69857f3725e05b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


## Model Selection - Flan T5 Base

Tokenizer and Model have been loaded specifically for sequence-to-sequence tasks, utilizing the "google/flan-t5-base" model from the Hugging Face

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

## Load dataset
Loading a Summarizing Dataset "knkarthick/dialogsum" Using  HuggingFace datasets Library

In [2]:
from datasets import load_dataset

# Load dataset from the hub
dataset_id = "knkarthick/dialogsum"
dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

# Train dataset size: 12460
# Test dataset size: 1500

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-d7c54319f1f4a15f/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Train dataset size: 12460
Test dataset size: 1500


**Train dataset size: 12460**


**Test dataset size: 1500**

### Lets checkout an example of the dataset.

In [ ]:
from random import randrange

sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"Dialogue: \n{sample['dialogue']}\n---------------")
print(f"Summary: \n{sample['summary']}\n---------------")

dialogue: 
#Person1#: Are you feeling all right, Mary? You look a little tired.
#Person2#: My brother and I had a birthday for my sister last night. So I didn't have much sleep.
#Person1#: Where did you have the party?
#Person2#: It was at my uncle's house.
#Person1#: Did you go back home very late?
#Person2#: Not really. But when I was home, I had to start typing a literature paper that was due first thing in the morning.
#Person1#: Couldn't you do it today?
#Person2#: No, our professor said she wouldn't accept any late papers.
#Person1#: Then how did you manage it?
#Person2#: Well, I just turned my paper in, and now I'm going home to bed. See you!
---------------
summary: 
Mary had a birthday party for her sister last night and stayed up late typing a literature paper.
---------------


### Pre-processing

Before we can start training we need to preprocess our data. Abstractive Summarization is a text2text-generation task. This means our model will take a text as input and generate a summary as output. For this we want to understand how long our input and output will be able to efficiently batch our data.

In [ ]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/13960 [00:00<?, ? examples/s]

Max source length: 512


Map:   0%|          | 0/13960 [00:00<?, ? examples/s]

Max target length: 277


The "preprocess_function" function is designed to manage tokenization and padding for both the entire chat and the summarization text

In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/12460 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['topic', 'input_ids', 'attention_mask', 'labels']


## Fine-tune, post-process and evaluate FLAN-T5




### Evaluation metrics function
Before starting training process, it's essential to have a evaluation metrics and function in place to display evaluation metrics for each iteration, providing ongoing feedback on the model's performance

In [ ]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Data loader
*Before* we can start training we need to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library.

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

### trainer instance
*The* last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training.

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

### Evaluate model before training

In [ ]:
trainer.evaluate()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.9551118612289429,
 'eval_rouge1': 24.6078,
 'eval_rouge2': 7.7833,
 'eval_rougeL': 21.2052,
 'eval_rougeLsum': 22.0787,
 'eval_gen_len': 14.144,
 'eval_runtime': 378.7294,
 'eval_samples_per_second': 3.961,
 'eval_steps_per_second': 1.98}

### Let's start training
We can start our training by using the `train` method of the `Trainer`.

In [ ]:
# Start training
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,1.090400,1.211200,39.490000,15.303000,33.349000,35.178500,18.850000
2,0.987300,1.200883,40.057900,16.418400,34.376700,35.877400,18.882000
3,0.900200,1.199109,39.979200,16.538100,34.307500,35.877400,18.844000
4,0.858800,1.224937,39.550000,16.126100,34.148500,35.694600,18.848000
5,0.803200,1.238433,39.468900,16.215100,33.959500,35.597200,18.890000


TrainOutput(global_step=31150, training_loss=0.9402237411180622, metrics={'train_runtime': 19484.741, 'train_samples_per_second': 3.197, 'train_steps_per_second': 1.599, 'total_flos': 4.26603830575104e+16, 'train_loss': 0.9402237411180622, 'epoch': 5.0})

Nice, we have trained our model. 🎉 Lets run evaluate the best model again on the test set.


### Evaluate model after training

In [ ]:
trainer.evaluate()

{'eval_loss': 1.199109435081482,
 'eval_rouge1': 39.9792,
 'eval_rouge2': 16.5381,
 'eval_rougeL': 34.3075,
 'eval_rougeLsum': 35.8774,
 'eval_gen_len': 18.844,
 'eval_runtime': 523.3275,
 'eval_samples_per_second': 2.866,
 'eval_steps_per_second': 1.433,
 'epoch': 5.0}

The best score we achieved is an `rouge1` score of `39.9`.


## Run Inference

Now we have a trained model, we can use it to run inference. We will use the `pipeline` API from transformers and a `test` example from our dataset.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# from transformers import pipeline
# from random import randrange
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

index =200
# load model and tokenizer from huggingface hub with pipeline
model_id="google/flan-t5-base"
trained_model = "/content/drive/MyDrive/flant5/checkpoint31150"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(trained_model)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# select a random test sample
sample = dataset['test'][index]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"human summary: \n{sample['summary']}\n---------------")

# summarize dialogue
prompt = f"""
summarize:

{sample}

"""

inputs = tokenizer(prompt, return_tensors='pt')
res = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

print(f"TRAINED MODEL SUMMARY:\n{res}")

(…)-base/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)flan-t5-base/resolve/main/tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

(…)ase/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

dialogue: 
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.
---------------
human summary: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.
---------------
TRAINED MODEL SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardwa

We have combined both the test and train datasets. In the aforementioned sample, the generated and human summaries are identical

In [9]:
#Lets try a sample from validation dataset

index =0

# select a random test sample
sample = dataset['validation'][index]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"human summary: \n{sample['summary']}\n---------------")

# summarize dialogue
prompt = f"""
summarize:

{sample}

"""

inputs = tokenizer(prompt, return_tensors='pt')
res = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

print(f"TRAINED MODEL SUMMARY:\n{res}")

dialogue: 
#Person1#: Hello, how are you doing today?
#Person2#: I ' Ve been having trouble breathing lately.
#Person1#: Have you had any type of cold lately?
#Person2#: No, I haven ' t had a cold. I just have a heavy feeling in my chest when I try to breathe.
#Person1#: Do you have any allergies that you know of?
#Person2#: No, I don ' t have any allergies that I know of.
#Person1#: Does this happen all the time or mostly when you are active?
#Person2#: It happens a lot when I work out.
#Person1#: I am going to send you to a pulmonary specialist who can run tests on you for asthma.
#Person2#: Thank you for your help, doctor.
---------------
human summary: 
#Person2# has trouble breathing. The doctor asks #Person2# about it and will send #Person2# to a pulmonary specialist.
---------------
TRAINED MODEL SUMMARY:
#Person2# has trouble breathing lately and #Person1# asks about #Person2#'s symptoms. #Person2# will see a pulmonary specialist.
